In [ ]:
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

Функция ```prepare_mask``` сдвигает матрицу перехода ```mask``` на ```i``` элементов по вертикали и на ```j``` элементов по горизонтали относительно левого верхнего края. После выполнения функции элемент ```pivot``` всегда окажется на позиции ```(i, j)```

In [ ]:
def prepare_mask(mask, pivot, i, j, n, m):
    mask_ = mask
    if i < pivot[0]:
        mask_ = mask_[(pivot[0] - i):]
    if n - i - 1 < n - pivot[0] - 1:
        mask_ = mask_[:(pivot[0] + n - i)]
    if j < pivot[1]:
        mask_ = mask_[:, (pivot[1] - j):]
    if m - j - 1 < m - pivot[1] - 1:
        mask_ = mask_[:, :(pivot[1] + m - j)]
    pad_up = max(0, i - pivot[0])
    pad_down = max(0, n - pad_up - mask_.shape[0])
    pad_left = max(0, j - pivot[1])
    pad_right = max(0, m - pad_left - mask_.shape[1])
    mask_ = np.pad(mask_, ((pad_up, pad_down), (pad_left, pad_right)), 'constant')
    return mask_

##### С использованием современных технологий, центр управления способен отслеживать синоптические данные в режиме реального времени. Применяя информации о параметрах ветра в районе проведения поисковой операции становится возможным предсказать вероятности нахождения оторвавшейся льдины в каждой из областей. В данном случае в качестве упрощения в модели берётся фиксированный квант времени, за который вероятности перемещения льдины заранее известны

Функция ```iteration``` производит одну итерацию применения матрицы перехода ```mask``` к каждому элементу массива вероятностей ```prob```

In [ ]:
def iteration(prob, mask, pivot, n, m):
    prob_ = np.zeros((n, m))
    for i in range(n):
        for j in range(m):
            prob_ += prob[i][j] * prepare_mask(mask, pivot, i, j, n, m)
    return prob_

##### В моделе предусмотрено введение в систему дополнительной информации с целью уточнения и корректировки уже просчитанных вероятностей. Предоставляется интерфейс ввода вероятностей нахождения льдины в каждой из областей. Это мотивировано спецификой работы нейронных сетей распознавания образов - на выходе модели мы всегда получаем вероятность обнаружения объекта. Таким образом, на практике можно использовать любй доступный беспилотный летательный аппарат с возможностью анализа входного видеопотока с наружней камеры для определения вероятности нахождения льдины с людьми в той области, которая в данный момент просматривается БЛА. Данный подход также мотивирован неидеальностью условий, в которых человек производит наблюдения. Даже в случае задействования спасательного вертолета с наблюдателем, погодные условия (туман, осадки...), высота, острота зрения, посторонние объекты на льдинах (тёмные камни, животные...) и прочие факторы могут сказаться на уверенности спасателя в обнаружении нужной льдины с людьми

Функция ```input_information``` возвращает массив, заполненный априорными вероятностями гипотез

In [ ]:
def input_information(n, m):
    info = np.full((n, m), None)
    return info

Функция ```clarify``` использует формулу Байеса для вычисления апостериорных вероятностей для каждого из элементов массива вероятностей ```prob``` используя априорные вероятности ```info```

In [ ]:
def clarify(prob, info, n, m):
    prob_ = prob
    for i in range(n):
        for j in range(m):
            if info[i][j] is not None:
                prob_[i][j] *= info[i][j]
    prob_ /= np.sum(prob_)
    return prob_

##### Визуальное отображение результатов вычислений важно для принятия решений спасательным центром ввиду абстрагирования данной модели от различных комплексных факторов, таких как риск проведения спасательной операции, особенности географии местности, оперативные данные и пр.

Функция ```plot_probability``` графически отображает массив вероятностей ```prob```

In [ ]:
def plot_probability(prob, n, m):
    plt.xlim(0, m - 1)
    plt.ylim(0, n - 1)
    plt.imshow(prob)

Вводные данные включают в себя:  
```n, m``` - размер района поиска в проекции, разделён на одинаковые области  
```prob``` - район поиска, заполненный вероятностями нахождения нужной льдины в каждой из областей  
```pivot``` - элемент матрицы перехода, совпадающий при наложении с переходящим элементом  
```mask``` - матрица перехода. Вероятности перехода льдины из данной области в соответствующие области относительно ```pivot```  
```wind``` - направление ветра. Смещение матрицы перехода относительно ```pivot```  

In [ ]:
n = 100
m = 200
prob = np.zeros((n, m))
prob[50][50] = 1
mask = np.array([[0.1, 0.1, 0.1], [0.1, 0.2, 0.1], [0.1, 0.1, 0.1]])
pivot = (1, 1)

In [ ]:
for i in tqdm(range(10)):
    prob = iteration(prob, mask, pivot, n, m)

In [ ]:
prob = clarify(prob, input_information(n, m), n, m)

In [ ]:
plot_probability(prob, n, m)